In [242]:
import nltk
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import natasha
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from sklearn.metrics import accuracy_score
import spacy
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
from nltk import word_tokenize
import en_core_web_sm
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

In [243]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

ImportError: cannot import name 'Segmenter' from 'natasha' (/home/marynepo/anaconda3/envs/py37/lib/python3.7/site-packages/natasha/__init__.py)

Тексты сложные для разметки, поскольку:
    1) Русский: конверсия (лучше), аббр. (ВШЭ), похожие формы у разных частей речи (кр. прич. vs кр. прил.), заимств. (лонгслив), разговорные (когдатошний)
    2) Англ: конверсия (water, repeat), сленг(do, obvs), аббревиатура (OP)

In [176]:
ru_txt = 'ВШЭ_N была_V создана_V в_PP конце_N прошлого_ADJ века_N. Он_PN был_V красив_ADJ. Я_PN сделаю_V это_PN лучше_ADV тебя_PN. Этот_PN вариант_N лучше_ADJ предыдущего_ADJ. Я_PN пойду_V в_PP ванную_N. Ты_PN пойдешь_V в_PP ванную_ADJ комнату_N? Я_PN встретил_V свою_PN когдатошнюю_ADJ знакомую_N Бетти_N. У_PP него_PN был_V знакомый_ADJ голос_N. Петровы_N и_C Муравьев_N, подойдите_V сюда_ADV. Теперь_ADV можно_V и_C почилить_V. Вань_N, быстро_ADV положи_V лонгслив_N на_PP стол_N. Я_PN вывел_V своих_PN гончих_N на_PP прогулку_N. На_PP каникулах_N доделайте_V пятый_NUM номер_N. Хорошо_I, я_PN это_PN сделаю_V. Таков_PN твой_PN ответ_N? Эх_I, это_P совсем_ADV не_P жиза_N. Гугл_N подсказал_V, что_C дисплей_N телефона_N состоит_V из_PP жидкокристаллических_ADJ ячеек_N, рассеивателей_N подсветки_N, шлейфа_N с_PP коннектором_N и_C закаленного_ADJ стекла_N. Предложение_N, добивающее_V этот_PN странный_ADJ текст_N до_PP ста_NUM слов_N.'
eng_txt = "I_PN drink_V water_N and_C water_N the_ART plants_N every_PN day_N. I_PN should_V jolly_ADV well_ADV think_V so_ADV! There_T was_V a_ART large_ADJ increase_N in_PP our_PN sales_N numbers_N. How_ADV can_V I_PN increase_V my_PN income_N? Robert_N aced_V his_PN physics_N exam_N! Should_V we_PN refund_V their_PN money_N? She_PN asked_V for_PP a_ART refund_N because_C the_ART product_N was_V defective_ADJ. Are_V you_PN going_V to_PP her_PN birthday_N do_N next_ADJ week_N? Well_I, he_PN must_V be_V nevertiree_N. This_PN situation_N is_V a_ART repeat_N of_PP what_PN happened_V last_ADJ year_N. Could_V you_PN repeat_V that_PN please_I? I_PN mean_V, obvs_ADV everyone_PN has_V played_V their_PN part_N, even_P Anya_N, but_C London_N is_V so_P back_ADJ? OP_N said_V nothing_PN about_PP it_PN."

In [237]:
dic_tag = {'N': ['S', 'NOUN', 'NN', 'NNS', 'NNP', 'NNPS', 'PROPN'], 'V': ['V', 'PRP$', 'VB', 'VBZ', 'VBG', 'VERB', 'INFN', 'PRTF', 'VBN', 'PRTS', 'GRND', 'AUX', 'VBP', 'MD', 'VBD'], 'PP': ['PR', 'PREP', 'PART', 'ADP', 'IN', 'TO'], 'ADJ': ['A', 'ADJF', 'ADJS', 'JJ'], 'ADV': ['ADV', 'ADVPRO', 'ADVB', 'PRED', 'ADP', 'RB', 'WRB'], 'PN': ['APRO', 'SPRO', 'NPRO', 'PRON', 'DET', 'PRP', 'WP', 'PRP$', 'DT'], 'NUM': ['NUM', 'ANUM', 'NUMR'], 'C': ['CONJ', 'CC', 'CCONJ', 'SCONJ'], 'I': ['INTJ', 'UH'], 'P': ['PRCL', 'PART', 'RB'], 'ART': ['DET', 'ART', 'DT'], 'T': ['DET', 'EX']}

In [189]:
def get_text_tags(text):
    
    tags = re.findall('_([A-Z]+)', text)
    clean_txt = re.sub('_[A-Z]+', '', text)
    
    return tags, clean_txt

In [190]:
ru_tg, ru_t = get_text_tags(ru_txt)
ru_cltxt = re.sub('[.,:?!]', '', ru_t)
pr_tags = ['']*len(ru_tg)
ru_words = ru_cltxt.split()

en_tg, en_t = get_text_tags(eng_txt)
en_cltxt = re.sub('[.,:?!]', '', en_t)
en_words = en_cltxt.split()

Pymorphy

In [99]:
m = MorphAnalyzer()
for i in range(len(ru_tg)):
    t = str(m.parse(ru_words[i])[0].tag.POS)
    if t in dic_tag[ru_tg[i]]:
        pr_tags[i] = ru_tg[i]
    else:
        pr_tags[i] = t
print("Accuracy пайморфи: %.4f" % accuracy_score(pr_tags, ru_tg))

Accuracy пайморфи: 0.8039


Mystem

In [100]:
ms = Mystem()
ana = ms.analyze(ru_t)
pr_tags = ['']*len(ru_tg)
ts = []
for word in ana:
    if 'analysis' in word:
        if len(word['analysis']) > 0:
            gr = word['analysis'][0]['gr']
            t = gr.split('=')[0].split(',')[0]
        else:
            t = 'none'
        ts.append(t)
            
for i in range(len(ru_tg)):
    if ts[i] in dic_tag[ru_tg[i]]:
        pr_tags[i] = ru_tg[i]
    else:
        pr_tags[i] = ts[i]
        
print("Accuracy майстема: %.4f" % accuracy_score(pr_tags, ru_tg))

Accuracy майстема: 0.9412


Natasha

In [244]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)
markup = next(morph.map(ru_words))

for i in range(len(markup.tokens)):
    if markup.tokens[i].pos in dic_tag[en_tg[i]]: #у меня небольшие трабблы с наташей, поэтому не могу проверить, так ли достается pos
        pr_tags[i] = en_tg[i]
    else:
        pr_tags[i] = markup.tokens[i].pos
            
print("Accuracy SpaCy: %.4f" % accuracy_score(pr_tags, en_tg))   

FileNotFoundError: [Errno 2] No such file or directory: 'navec_news_v1_1B_250K_300d_100q.tar'

SpaCy

In [203]:
nlp = en_core_web_sm.load()
doc = nlp(en_cltxt)
pr_tags = [''] * len(doc)

for i in range(len(doc)):
    if doc[i].pos_ in dic_tag[en_tg[i]]:
        pr_tags[i] = en_tg[i]
    else:
        pr_tags[i] = doc[i].pos_
            
print("Accuracy SpaCy: %.4f" % accuracy_score(pr_tags, en_tg))

Accuracy SpaCy: 0.9192


Flair

In [210]:
splitter = SegtokSentenceSplitter()
sentences = splitter.split(en_t)
tagger = SequenceTagger.load('pos')
tagger.predict(sentences)

2020-10-18 22:35:59,889 loading file /home/marynepo/.flair/models/en-pos-ontonotes-v0.5.pt


In [238]:
pr_tags = []
for s in sentences:
    s = s.to_tagged_string()
    s = re.sub('<[.,]>', '', s)
    tgs = re.findall('<(.+?)>', s)
    pr_tags += tgs
    
for i in range(len(en_tg)):
    if pr_tags[i] in dic_tag[en_tg[i]]:
        pr_tags[i] = en_tg[i]
    
print("Accuracy Flair: %.4f" % accuracy_score(pr_tags, en_tg))

Accuracy Flair: 0.9192


NLTK

In [239]:
pr_tags = [''] * len(en_tg)
ttext = nltk.pos_tag(word_tokenize(en_cltxt))
for i in range(len(ttext)):
    if ttext[i][1] in dic_tag[en_tg[i]]:
        pr_tags[i] = en_tg[i]
    else:
        pr_tags[i] = ttext[i][1] 
        
print("Accuracy NLTK: %.4f" % accuracy_score(pr_tags, en_tg))

Accuracy NLTK: 0.8687


Не успела написать функцию, но взяла бы, во-первых прил. + сущ. (мне кажется, что (если говорить про отзывы) именно прил. часто имеют яркок-выраженый позитив. или негативную окраску). И также посмотрела бы на частицы (so, так) + нар./прил, кажется, часто такие сочетания будут не нейтральными, а частицы будут усилительными.